In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
train_df = pd.read_csv("./Data/train.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 12 columns):
datetime         207 non-null object
y                207 non-null int64
week             207 non-null object
soldout          207 non-null int64
name             207 non-null object
kcal             166 non-null float64
remarks          21 non-null object
event            14 non-null object
payday           10 non-null float64
weather          207 non-null object
precipitation    207 non-null object
temperature      207 non-null float64
dtypes: float64(3), int64(2), object(7)
memory usage: 19.5+ KB


In [3]:
test_df = pd.read_csv("./Data/test.csv")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
datetime         40 non-null object
week             40 non-null object
soldout          40 non-null int64
name             40 non-null object
kcal             36 non-null float64
remarks          7 non-null object
event            3 non-null object
payday           2 non-null float64
weather          40 non-null object
precipitation    40 non-null object
temperature      40 non-null float64
dtypes: float64(3), int64(1), object(7)
memory usage: 3.5+ KB


In [4]:
#rain_test_df = train_df.append(test_df)
train_test_df = pd.concat([train_df, test_df], axis=0)
train_test_df = train_test_df.reindex(columns=train_df.columns)
train_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 39
Data columns (total 12 columns):
datetime         247 non-null object
y                207 non-null float64
week             247 non-null object
soldout          247 non-null int64
name             247 non-null object
kcal             202 non-null float64
remarks          28 non-null object
event            17 non-null object
payday           12 non-null float64
weather          247 non-null object
precipitation    247 non-null object
temperature      247 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 25.1+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [6]:
train_test_df[190:210]

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
190,2014-9-4,54.0,木,1,チキンステーキ・きのこソース,405.0,NaN,NaN,NaN,曇,0,26.5
191,2014-9-5,43.0,金,0,メンチカツ,380.0,NaN,NaN,NaN,晴れ,--,30.2
192,2014-9-8,68.0,月,1,鶏肉の山賊焼き,385.0,NaN,NaN,NaN,曇,--,23.1
193,2014-9-9,63.0,火,0,ハンバーグデミソース,460.0,NaN,NaN,NaN,晴れ,--,26.8
194,2014-9-10,54.0,水,0,ぶりレモンペッパー焼き,450.0,NaN,NaN,1.0,曇,0,25.0
195,2014-9-11,53.0,木,0,手作りチキンカツ,385.0,NaN,NaN,NaN,曇,--,21.9
196,2014-9-12,115.0,金,0,ポークカレー,NaN,お楽しみメニュー,NaN,NaN,晴れ,--,27.3
197,2014-9-16,56.0,火,0,チーズメンチカツ,438.0,NaN,NaN,NaN,晴れ,--,28.8
198,2014-9-17,49.0,水,0,チキンフリカッセ,430.0,料理長のこだわりメニュー,ママの会,NaN,曇,--,25.0
199,2014-9-18,46.0,木,0,カレイ唐揚げ 甘酢あん,395.0,NaN,NaN,NaN,曇,--,24.1


In [7]:
train_test_df.to_csv("B02.csv", index=False)

In [8]:
train_test_df = pd.read_csv("./B02.csv")
train_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247 entries, 0 to 246
Data columns (total 12 columns):
datetime         247 non-null object
y                207 non-null float64
week             247 non-null object
soldout          247 non-null int64
name             247 non-null object
kcal             202 non-null float64
remarks          28 non-null object
event            17 non-null object
payday           12 non-null float64
weather          247 non-null object
precipitation    247 non-null object
temperature      247 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 23.2+ KB


In [9]:
train_test_df.head(10)

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90.0,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,101.0,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,118.0,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,120.0,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,130.0,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1
5,2013-11-25,135.0,月,1,鶏の唐揚,NaN,NaN,NaN,NaN,曇,--,14.6
6,2013-11-26,145.0,火,0,豚のスタミナ炒め,NaN,NaN,NaN,NaN,快晴,--,17.9
7,2013-11-27,140.0,水,1,ボローニャ風カツ,NaN,NaN,NaN,NaN,晴れ,--,14.7
8,2013-11-28,151.0,木,0,ハンバーグ,NaN,NaN,NaN,NaN,薄曇,--,17.7
9,2013-11-29,116.0,金,0,タルタルinソーセージカツ,NaN,NaN,NaN,NaN,快晴,--,12.1
